# 01_prepare_data / data check
Quick sanity checks for the raw DWD station files (wind + cloudiness). Adjust paths if you run the notebook elsewhere.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE_DIR = Path(r'U:\\Abt02\\Ref23\\Daten\\LQ-Modellierung\\06_Modellierung\\04_WINMiskam\\02_Meteorologie')
DATA_DIR = BASE_DIR / "data" / "raw"

wind_candidates = list((DATA_DIR / "wind").glob("03379_*stadt.txt"))
if not wind_candidates:
    raise FileNotFoundError("No wind file matching 03379_*stadt.txt found in data/raw/wind")
WIND_PATH = wind_candidates[0]

CLOUD_PATH = DATA_DIR / "cloudiness" / "produkt_n_stunde_19790101_20241231_03379.txt"

print("wind file:", WIND_PATH)
print("cloud file:", CLOUD_PATH)

try:
    # Prefer IANA zone name; may be missing on some Windows setups
    _MEZ_TZ = 'Europe/Berlin'
    pd.Timestamp('2000-01-01', tz=_MEZ_TZ)
except Exception:
    _MEZ_TZ = 'CET'

# Target year for processing
YEAR = 2009




wind file: U:\Abt02\Ref23\Daten\LQ-Modellierung\06_Modellierung\04_WINMiskam\02_Meteorologie\data\raw\wind\03379_Muenchen_stadt.txt
cloud file: U:\Abt02\Ref23\Daten\LQ-Modellierung\06_Modellierung\04_WINMiskam\02_Meteorologie\data\raw\cloudiness\produkt_n_stunde_19790101_20241231_03379.txt


In [2]:
# Wind: load, clean, timestamp
wind = pd.read_csv(WIND_PATH, sep=';', encoding='latin1')
wind.columns = [c.strip() for c in wind.columns]
wind = wind.rename(columns={'F': 'wind_speed_ms', 'D': 'wind_dir_deg'})
wind_speed_rounded = wind['wind_speed_ms'].round(1)
if not np.allclose(wind['wind_speed_ms'], wind_speed_rounded, equal_nan=True):
    wind['wind_speed_ms'] = wind_speed_rounded
# Raw wind timestamps are UTC; parse and keep tz-aware
wind['timestamp'] = pd.to_datetime(
    wind['MESS_DATUM'].astype(str), format='%Y%m%d%H', errors='coerce', utc=True
)
wind.head()


,STATIONS_ID,MESS_DATUM,QN_3,wind_speed_ms,wind_dir_deg,eor,timestamp
0,3379,1985010100,10,1.8,220,eor,1985-01-01 00:00:00+00:00
1,3379,1985010101,10,1.0,230,eor,1985-01-01 01:00:00+00:00
2,3379,1985010102,10,1.6,230,eor,1985-01-01 02:00:00+00:00
3,3379,1985010103,10,1.5,170,eor,1985-01-01 03:00:00+00:00
4,3379,1985010104,10,1.8,180,eor,1985-01-01 04:00:00+00:00


In [3]:
# Timestamps stay in UTC; just sort
wind = wind.sort_values("timestamp").reset_index(drop=True)
wind.head()


,STATIONS_ID,MESS_DATUM,QN_3,wind_speed_ms,wind_dir_deg,eor,timestamp
0,3379,1985010100,10,1.8,220,eor,1985-01-01 00:00:00+00:00
1,3379,1985010101,10,1.0,230,eor,1985-01-01 01:00:00+00:00
2,3379,1985010102,10,1.6,230,eor,1985-01-01 02:00:00+00:00
3,3379,1985010103,10,1.5,170,eor,1985-01-01 03:00:00+00:00
4,3379,1985010104,10,1.8,180,eor,1985-01-01 04:00:00+00:00


In [4]:
# Cloudiness: load, clean, timestamp
cloud = pd.read_csv(CLOUD_PATH, sep=';', encoding='latin1')
cloud.columns = [c.strip() for c in cloud.columns]
cloud = cloud.rename(columns={'V_N': 'cloud_cover_oktas', 'V_N_I': 'cloud_cover_flag', 'QN_8': 'cloud_qn'})
cloud['timestamp'] = pd.to_datetime(
    cloud['MESS_DATUM'].astype(str), format='%Y%m%d%H', errors='coerce', utc=True
)
# Keep cloud timestamps in UTC (no conversion)
cloud['timestamp'] = pd.DatetimeIndex(cloud['timestamp'])
cloud = cloud.sort_values("timestamp").reset_index(drop=True)
cloud.head()


,STATIONS_ID,MESS_DATUM,cloud_qn,cloud_cover_flag,cloud_cover_oktas,eor,timestamp
0,3379,1979010106,1,P,8,eor,1979-01-01 06:00:00+00:00
1,3379,1979010118,1,P,8,eor,1979-01-01 18:00:00+00:00
2,3379,1979010206,1,P,3,eor,1979-01-02 06:00:00+00:00
3,3379,1979010218,1,P,1,eor,1979-01-02 18:00:00+00:00
4,3379,1979010306,1,P,7,eor,1979-01-03 06:00:00+00:00


In [5]:
# Merge on station + timestamp (normalize to UTC tz-aware)
wind['timestamp'] = pd.to_datetime(wind['timestamp'], errors='coerce', utc=True)
cloud['timestamp'] = pd.to_datetime(cloud['timestamp'], errors='coerce', utc=True)

merged = wind.merge(
    cloud[["STATIONS_ID", "timestamp", "cloud_qn", "cloud_cover_oktas", "cloud_cover_flag"]],
    on=["STATIONS_ID", "timestamp"],
    how='left',
)
merged.head()


,STATIONS_ID,MESS_DATUM,QN_3,wind_speed_ms,wind_dir_deg,eor,timestamp,cloud_qn,cloud_cover_oktas,cloud_cover_flag
0,3379,1985010100,10,1.8,220,eor,1985-01-01 00:00:00+00:00,1.0,8.0,P
1,3379,1985010101,10,1.0,230,eor,1985-01-01 01:00:00+00:00,NaN,NaN,NaN
2,3379,1985010102,10,1.6,230,eor,1985-01-01 02:00:00+00:00,NaN,NaN,NaN
3,3379,1985010103,10,1.5,170,eor,1985-01-01 03:00:00+00:00,1.0,8.0,P
4,3379,1985010104,10,1.8,180,eor,1985-01-01 04:00:00+00:00,NaN,NaN,NaN


In [6]:
# Filter target year and export
import pandas as pd
from pathlib import Path

# Explicit defaults to avoid unbound warnings
BASE_DIR = Path(r'U:\\Abt02\\Ref23\\Daten\\LQ-Modellierung\\06_Modellierung\\04_WINMiskam\\02_Meteorologie')
YEAR = 2009
if 'merged' not in globals():
    raise RuntimeError("merged DataFrame is missing; run the upstream merging cells first.")

year_start = pd.Timestamp(f"{YEAR}-01-01 00:00:00", tz='UTC')
year_end = pd.Timestamp(f"{YEAR+1}-01-01 00:00:00", tz='UTC')
mask_year = (merged['timestamp'] >= year_start) & (merged['timestamp'] < year_end)
merged_year = merged.loc[mask_year].copy()
out_path = BASE_DIR / "data" / "processed" / f"merged_wind_cloud_{YEAR}.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
merged_year.to_csv(out_path, index=False)
print(out_path)
print(f"rows in {YEAR}:", len(merged_year))
display(merged_year.head())

U:\Abt02\Ref23\Daten\LQ-Modellierung\06_Modellierung\04_WINMiskam\02_Meteorologie\data\processed\merged_wind_cloud_2009.csv
rows in 2009: 8760


,STATIONS_ID,MESS_DATUM,QN_3,wind_speed_ms,wind_dir_deg,eor,timestamp,cloud_qn,cloud_cover_oktas,cloud_cover_flag
210059,3379,2009010100,10,1.5,280,eor,2009-01-01 00:00:00+00:00,3.0,5.0,P
210060,3379,2009010101,10,1.6,250,eor,2009-01-01 01:00:00+00:00,NaN,NaN,NaN
210061,3379,2009010102,10,2.0,240,eor,2009-01-01 02:00:00+00:00,NaN,NaN,NaN
210062,3379,2009010103,10,1.7,250,eor,2009-01-01 03:00:00+00:00,NaN,NaN,NaN
210063,3379,2009010104,10,1.9,230,eor,2009-01-01 04:00:00+00:00,NaN,NaN,NaN
